# Tarea 2

Mostramos la implementación:

In [ ]:
ENIGMA_ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ" # alfabeto
# diccionario, letra del alfabeto -> índice numérico del alfabeto
ch_ind = {ENIGMA_ALPHABET[i]: i for i in range(26)}
# tupla, índice numérico del alfabeto -> letra del alfabeto
ind_ch = tuple(ENIGMA_ALPHABET[i] for i in range(26))

# notches indicará las letras que se ven en la ventana antes de que al pulsar
# tecla pawl engache un notch, notches se usará en caso de los rotores
class CompData:
    def __init__(self, outputs, notches = None):
        self.outputs = outputs
        self.notches = notches

# Datos de rotores
ROTORS = {
    # 5 rotores extendidos en 1939, con un único notch
    '1': CompData('EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'Q'),
    '2': CompData('AJDKSIRUXBLHWTMCQGZNPYFVOE', 'E'),
    '3': CompData('BDFHJLCPRTXVZNYEIWGAKMUSQO', 'V'),
    '4': CompData('ESOVPZJAYQUIRHXLNFTGKDCMWB', 'J'),
    '5': CompData('VZBRGITYUPSDNHLXAWMJQOFECK', 'Z'),
    # 3 rotores adicionales añadidos por Kriegsmarine, tienen dos notches
    '6': CompData('JPGVOUMFYQBENHZRDKASXLICTW', 'ZM'),
    '7': CompData('NZJHGRCXMYSWBOUFAIVLPEKQDT', 'ZM'),
    '8': CompData('FKQHTLXOCBJSPDZRAMEWNIUYGV', 'ZM'),
    # Cuarto rotor (1942), usados exclusivamente por M4 con reflectores delgados
    'b': CompData('LEYJVCNIXWPBQMDRTAKZGFUHOS', None),
    'g': CompData('FSOKANUERHMBTIYCWLQPZXVGJD', None)
}

# Datos de reflectores
REFLECTORS = {
    # Reflectores iniciales Enigma I
    'B': CompData('YRUHQSLDPXNGOKMIEBFZCWVJAT'),
    'C': CompData('FVPJIAOYEDRZXWGCTKUQSBNMHL'),
    # Reflectores añadidos por Kriegsmarine que permiten espacio para
    # el cuarto rotor:
    'B Thin': CompData('ENKQAUYWJICOPBLMDXZVFTHRGS'),
    'C Thin': CompData('RDOBJNTKVEHMLFCWZAXGYIPSUQ')
}

In [ ]:
class Rotor:
    def __init__(self, outputs: str, ringst_chr: str, notches: str):
        """
        Inicialización.
        Params:
            outputs: string con la salidas que efectúa el rotor a cada letra
             del alfabeto dispuestas en orden
            ringst_char: letra que queda en primer contacto del rotor por el
             deslizamiento por ringstellung
            notches: string con letras visibles en ventanilla cuando una muesca
             está sobre pawl ("el trinquete")
        """
        ringst_offset = ch_ind[ringst_chr] # a índice numérico
        # tupla con los índices numéricos de las salidas de las conexiones
        self.connections = tuple(ch_ind[c] for c in outputs)
        # tupla inversa de connections para la transmisión después de reflector
        inv_connects_l = [None] * 26
        for i, c in enumerate(self.connections):
            inv_connects_l[c] = i
        self.inv_connections = tuple(inv_connects_l)

        # posiciones del rotor en las que un "trinquete" enganchará notch
        # (posiciones "deshaciendo ringstellung")
        self.p_notches_pos = None if notches is None else \
        tuple(ch_ind[n] - ringst_offset for n in notches)

        self.ringstellung = ringst_offset
        self.pos = 0 # posición inicial

    def set_window_position(self, pos: int):
        """
        Coloca el rotor en posición que deje al índice de letra pos
        visible en la ventanilla
        """
        # colocamos rotor en posición real, "deshaciendo ringstellung"
        self.pos = (pos - self.ringstellung) % 26

    def transmit(self, contact: int, before_reflector: bool) -> int:
        """
        Transmite señal a través del rotor
        """
        # contacto receptor de la señal en su posición de rotación
        r_contact = (contact + self.pos) % 26
        # contacto transmitido, relativo a la rotación del rotor
        t_contact_rel = self.connections[r_contact] if before_reflector \
        else self.inv_connections[r_contact]

        # contacto transmitido en posición absoluta
        t_contact_abs = (t_contact_rel - self.pos) % 26

        return t_contact_abs

    def rotate(self):
        """
        Rota el rotor
        """
        self.pos = (self.pos + 1) % 26 # rota actualizando posición

    def pawl_drops_into_notch(self) -> bool:
        """
        Comprobueba si el notch (muesca) está sobre los "trinquetes" (pawls) y
        por tanto será enganchado al pulsar tecla
        """
        # devuelve si notch será enganchado al pulsar tecla
        return self.pos in self.p_notches_pos


In [ ]:
class Reflector:
    def __init__(self, outputs: str):
        """
        Inicialización.
        Params:
            outputs: string con la salidas que efectúa el reflector a cada letra
             del alfabeto dispuestas en orden
        """
        # conexiones, tupla con los contactos de salida
        self.connections = tuple(ch_ind[c] for c in outputs)

    def transmit(self, contact: int) -> int:
        """
        Transmite señal a través del reflector
        """
        # devolvemos contacto al que transmite
        return self.connections[contact]


In [ ]:
class Plugboard:
    def __init__(self, swaps: str):
        """
        Inicialización.
        Params:
            swaps: string con parejas de letras conectadas en Plugboard
        """
        # diccionario que realiza los intercambios correspondientes por las
        # conexiones dadas en swaps
        cable_swaps = {ch_ind[p[i]]: ch_ind[p[(i+1)%2]] for p in swaps.split() for i in range(2)}
        # tupla con las salidas tras el plugboard, salidas por intercambio por
        # cable conectado y salidas que quedan igual por no haber cable (más
        # eficiente que comprobar si hay intercambio en cable_swaps)
        self.swaps = tuple(cable_swaps[i] if i in cable_swaps else i for i in range(26))

    def swap(self, c: int) -> int:
        """
        Da la salida determinada por la configuración del plugboard
        """
        # devuelve salida correspondiente
        return self.swaps[c]


In [ ]:
class EnigmaMachine:
    def __init__(self, pb_settings: str, sorted_rotors: str, ringstellung: str,\
     reflector: str):
        """
        Inicialización.
        Params:
            pb_settings: parejas de letras conectadas en Plugboard
            sorted_rotors: nombres de los rotores a usar vistos de izqda a dcha
             en la máquina
            ringstellung: letras en primer contacto de cada rotor tras
             deslizamientos por ringstellung
            reflector: nombre del reflector a usar
        """
        rotors_t = tuple(sorted_rotors) # tupla con los rotores a usar

        # Se crea tupla con los rotores
        rotors = tuple( Rotor(r_data.outputs, rr[1], r_data.notches) for rr in \
        zip(rotors_t, ringstellung) if (r_data := ROTORS[rr[0]]) )

        self.plugboard = Plugboard(pb_settings)
        self.rotors = rotors
        self.reflector = Reflector(REFLECTORS[reflector].outputs)

    def _rotate_rotors(self):
        """
        Rota los rotores que correspondan
        """
        # Rotores candidatos a rotar (todos menos el cuarto rotor)
        cand_rotors = self.rotors[-3:]
        # Crearemos tupla booleana para rotores que rotarán, el primero siempre
        # rota. Para los otros dos comprobamos si notch es enganchado
        # - primer rotor rota siempre (no tiene rotor a dcha)
        # - segundo rotor rota si el segundo pawl engancha notch de primer rotor
        #   o si tercer pawl engancha notch de segundo rotor
        # - tercer rotor rota si tercer pawl engancha notch de segundo rotor
        to_rotate2 = cand_rotors[-2].pawl_drops_into_notch()
        rotates = \
        (to_rotate2, cand_rotors[-1].pawl_drops_into_notch() or to_rotate2, 1)
        # Rotamos los rotores que corresponden rotar
        for i, r in enumerate(cand_rotors):
            if rotates[i]:
                r.rotate()

    def _transmission(self, c: int) -> int:
        """
        Efectúa la transmisión del índice de letra c a través de la máquina
        """
        # Intercambiamos por plugboard
        contact = self.plugboard.swap(c)
        # Transmitimos por los contactos de rotores de "derecha a izquierda"
        for r in self.rotors[::-1]:
            contact = r.transmit(contact, True)
        # Transmitimos a reflector
        contact = self.reflector.transmit(contact)
        # Transmitimos de vuelta por los rotores
        for r in self.rotors:
            contact = r.transmit(contact, False)
        # Devolvemos el intercambio por plugboard
        return self.plugboard.swap(contact)

    def grundstellung(self, win_pos_list: str):
        """
        Se colocan rotores en las posiciones que dejan los caracteres de
        win_pos_list visibles en ventanilla
        """
        for rotor, pos in zip(self.rotors, win_pos_list):
            rotor.set_window_position(ch_ind[pos])

    def encipher_chr(self, chr: str) -> str:
        """
        Cifra una letra pulsada
        """
        # Se rotan los rotores que correspondan
        self._rotate_rotors()
        # Se devuelve caracter cifrado
        return ind_ch[self._transmission(ch_ind[chr])]

    def encipher_text(self, text: str) -> str:
        """
        Cifra un texto letra a letra
        """
        enciphered_text = ''
        # Se cifra el texto letra a letra
        for c in text:
            enciphered_text += self.encipher_chr(c)
        # Se devuelve el texto cifrado
        return enciphered_text


La máquina M4 es compatible con la M3 usando reflector B delgado (B Thin) junto con cuarto rotor beta en posición A y ringstellung A, o reflector C con rotor gamma. Un ejemplo simulado (ficticio) para keysheet de Octubre 1944, en día 31 (http://users.telenet.be/d.rijmenants/pics/hires-wehrmachtkey-stab.jpg) :

In [ ]:
sender_machine = EnigmaMachine(
 pb_settings='KL IT FQ HY XC NP VZ JB SE OG',
 sorted_rotors='b451', 
 ringstellung='A'+ind_ch[21]+ind_ch[15]+ind_ch[16],
 reflector='B Thin'
       )
       
sender_machine.grundstellung('ALFD') # 'LFD' aleatorio, se enviará con el mensaje
msg_key = sender_machine.encipher_text('RLO') # 'RLO' aleatorio, msg_key se enviará con el mensaje

In [ ]:
msg_key

In [ ]:
sender_machine.grundstellung('A'+'RLO')

El emisor mandará el mensaje: "DIEVRAKETENTREFFENERFOLGREICH" -> Die V Raketen treffen erfolgreich (Los misiles V impactaron con éxito)

In [ ]:
mess='DIEVRAKETENTREFFENERFOLGREICH'

In [ ]:
enciphered_message = sender_machine.encipher_text(mess) # ciframos

Mostramos texto cifrado:

In [ ]:
enciphered_message

El mensaje se envía con grupo de 5 letras que indica Kenngruppen:

In [ ]:
'ZTJKM'+enciphered_message

El receptor ha de tener la misma configuración en la máquina (clave simétrica)

In [ ]:
receiver_machine = EnigmaMachine(
 pb_settings='KL IT FQ HY XC NP VZ JB SE OG',
 sorted_rotors='b451', 
 ringstellung='A'+ind_ch[21]+ind_ch[15]+ind_ch[16],
 reflector='B Thin'
       )

receiver_machine.grundstellung('ALFD')
un_msg_key = receiver_machine.encipher_text(msg_key)
un_msg_key

In [ ]:
receiver_machine.grundstellung('A'+un_msg_key) # ajusta rotores en posiciones correspondientes

Ahora descifra el mensaje sin los primeros cinco caracteres que indican Kenngruppen

In [ ]:
received_mess=receiver_machine.encipher_text(enciphered_message); received_mess

Que sería: "Die V Raketen treffen erfolgreich" (Los misiles V impactaron con éxito)

Podemos comprobar que es correcto

In [ ]:
received_mess == mess

Referencias:

http://users.telenet.be/d.rijmenants/en/enigmatech.htm

http://users.telenet.be/d.rijmenants/en/enigmaproc.htm

Nuestro simulador también nos hubiese permitido simular/operar como una M3 de la siguiente forma (abreviamos todos los pasos y comentarios para no ser redundantes):

In [ ]:
sender_machine = EnigmaMachine(
 pb_settings='KL IT FQ HY XC NP VZ JB SE OG',
 sorted_rotors='451', 
 ringstellung=ind_ch[21]+ind_ch[15]+ind_ch[16],
 reflector='B'
       )
       
sender_machine.grundstellung('LFD') # 'LFD' aleatorio, se enviará con el mensaje
msg_key = sender_machine.encipher_text('RLO') # 'RLO' aleatorio, msg_key se enviará con el mensaje
msg_key

In [ ]:
sender_machine.grundstellung('RLO')

In [ ]:
mess='DIEVRAKETENTREFFENERFOLGREICH'

In [ ]:
enciphered_message = sender_machine.encipher_text(mess); enciphered_message

El mensaje se envía con grupo de 5 letras que indica Kenngruppen: 'ZTJKM'+enciphered_message

Receptor:

In [ ]:
receiver_machine = EnigmaMachine(
 pb_settings='KL IT FQ HY XC NP VZ JB SE OG',
 sorted_rotors='451', 
 ringstellung=ind_ch[21]+ind_ch[15]+ind_ch[16],
 reflector='B'
       )

receiver_machine.grundstellung('LFD')
un_msg_key = receiver_machine.encipher_text(msg_key)
un_msg_key

In [ ]:
receiver_machine.grundstellung(un_msg_key)

In [ ]:
received_mess=receiver_machine.encipher_text(enciphered_message); received_mess

In [ ]:
received_mess == mess

# Tarea 3

Configuración de la máquina:

In [ ]:
machine = EnigmaMachine(
 pb_settings='AE BF CM DQ HU JN LX PR SZ VW',
 sorted_rotors='b568',
 ringstellung='EPEL',
 reflector='C Thin'
       )
       
machine.grundstellung('NAEM')

msg_key = machine.encipher_text('QEOB')

machine.grundstellung(msg_key)

In [ ]:
source_text = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT\
GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN\
VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH\
VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO\
TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH\
DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML\
RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"

Eliminamos los 2 primeros grupos de 4 letras y los dos últimos ya que están para verificar correcta recepción y no son parte del mensaje como tal. También eliminamos espacios pues no están en el alfabeto de la Máquina Enigma.

In [ ]:
ciphertext = source_text.replace("DUHF TETO", "").replace(" ","") 

Descifraremos ciphertext

In [ ]:
plaintext = machine.encipher_text(ciphertext)

Mostramos mensaje descifrado:

In [ ]:
plaintext

Comprobaremos, comparando con plaintextCryptoMuseum, que el mensaje descifrado es correcto

In [ ]:
plaintextCryptoMuseum = "KRKRALLEXXFOLGENDESISTSOFORTBEKANNTZUGEBENXXICHHABEFOLGELNBEBEFEHLERHALTENXXJANSTERLEDESBISHERIGXNREICHSMARSCHALLSJGOERINGJSETZTDERFUEHRERSIEYHVRRGRZSSADMIRALYALSSEINENNACHFOLGEREINXSCHRIFTLSCHEVOLLMACHTUNTERWEGSXABSOFORTSOLLENSIESAEMTLICHEMASSNAHMENVERFUEGENYDIESICHAUSDERGEGENWAERTIGENLAGEERGEBENXGEZXREICHSLEITEIKKTULPEKKJBORMANNJXXOBXDXMMMDURNHFKSTXKOMXADMXUUUBOOIEXKP"

In [ ]:
plaintext == plaintextCryptoMuseum

# Tarea 4

Para que el mensaje no pueda ser leído por intermediarios, este debe estar cifrado con una configuración de la máquina que solo conozcan emisor y receptor. Tras cifrar mediante esta configuración, se debería añadir un mensaje inicial que indique a quién va dirigido. Este mensaje inicial junto con el resto de mensaje confidencial ya cifrado, se cifrarán conjuntamente con la configuración que corresponda para que pueda ser descifrado por el intermediario receptor y pueda saber a quién va dirigido.

Vamos a simular que el oficial de tierra del alto mando Friedrich (que tiene una M3) quiere enviar un mensaje confidencial al capitán de submarino Berthold. Para comprobar que una M4 puede descifrar lo de una M3 supondremos que en el submarino tienen una M4

El mensaje confidencial, en el contexto de comunicaciones previas, será: "ICHVERTRAUEALLMEINENMANNERN" -> "Ich vertraue all meinen Männern" (Confío en todos mis hombres)

En nuestra simulación, Fiedrich y Berthold disponen de hoja confidencial (secreta y solo conocida por ellos) para oficiales o altos mandos en la que hay distintas configuraciones en plugboard dependiendo del día, y un grundstellung asignado diferente para cada oficial.

Vamos a simular cómo sería el mensaje entre altos mandos: del oficial Friedrich al capitán Berthold

El oficial Friedrich coloca el cableado del plugboard correspondiente al día, y su grundstellung. El resto de configuración la correspondiente al keysheet conocido por todos. Simulamos día 29 de octubre de 1944

In [ ]:
machine_oficial_F = EnigmaMachine(
 pb_settings='AO DQ CY HU JN LX PR SZ VW FI', # la del keysheet de oficiales
 sorted_rotors='254',
 ringstellung=ind_ch[19]+ind_ch[9]+ind_ch[24],
 reflector='B'
       )
       
machine_oficial_F.grundstellung('GCA') # grundstellung de keysheet de oficiales

In [ ]:
official_F_message = "ICHVERTRAUEALLMEINENMANNERN"

In [ ]:
offF_ciphered_mess = machine_oficial_F.encipher_text(official_F_message)

Mensaje cifrado

In [ ]:
offF_ciphered_mess

Ahora el oficial Friedrich pasa ese mensaje cifrado a alguien de rango inferior que se encargará de enviar el mensaje con la configuración conocida entre los de su rango

In [ ]:
machine_rangoinferior1 = EnigmaMachine(
 pb_settings='ZU HL CQ WM OA PY EB TR DN VI',
 sorted_rotors='254',
 ringstellung=ind_ch[19]+ind_ch[9]+ind_ch[24],
 reflector='B'
       )

machine_rangoinferior1.grundstellung('LFD') # 'SPS' aleatorio, se enviará con el mensaje
msg_key = machine_rangoinferior1.encipher_text('MIC') # 'RLO' aleatorio, msg_key se enviará con el mensaje
msg_key

In [ ]:
machine_rangoinferior1.grundstellung('MIC')

Recibe el mensaje cifrado y añade "OFFIZIERBERTHOLDVONKAPITANFRIEDRICH" (indica oficial al que va dirigido y de quién). El mensaje que cifrará el de rango inferior es

In [ ]:
message = "OFFIZIERBERTHOLDVONKAPITANFRIEDRICH" + offF_ciphered_mess; message

Ahora lo cifra todo con la configuración del keysheet conocido por todos

In [ ]:
ciphered_message = machine_rangoinferior1.encipher_text(message)

Mensaje cifrado a enviar:

In [ ]:
ciphered_message

El mensaje se envía con grupo de 5 letras que indica Kenngruppen: 'RDOID'+ciphered_message

In [ ]:
'RDOID'+ciphered_message

Ahora alguien del mismo rango en el submarino recibe el mensaje y lo descifra con la misma configuración de máquina, (ignora Kenngruppen)

In [ ]:
machine_rangoinferior2 = EnigmaMachine(
 pb_settings='ZU HL CQ WM OA PY EB TR DN VI',
 sorted_rotors='b254',
 ringstellung='A'+ind_ch[19]+ind_ch[9]+ind_ch[24],
 reflector='B Thin'
       )

machine_rangoinferior2.grundstellung('ALFD')
un_msg_key = machine_rangoinferior2.encipher_text(msg_key)
un_msg_key

In [ ]:
machine_rangoinferior2.grundstellung('A'+un_msg_key)

In [ ]:
deciphered_message=machine_rangoinferior2.encipher_text(ciphered_message)

In [ ]:
deciphered_message

Vemos que comienza por OFFIZIERBERTHOLDVONKAPITANFRIEDRICH, el rango inferior sabrá que debe entregar el resto del mensaje al capitán Berthold. El mensaje para Berthold será

In [ ]:
capB_message = deciphered_message.replace('OFFIZIERBERTHOLDVONKAPITANFRIEDRICH','')

In [ ]:
capB_message

In [ ]:
capB_message == offF_ciphered_mess

Ahora puede obtener su mensaje el capitán Berthold. Este consultará su hoja confidencial de altos mandos para configurar el plugboard correspondiente al día y los rotores correspondientes a los del oficial Friedrich

In [ ]:
# misma configuración que oficial Friedrich
machine_capitan_B = EnigmaMachine(
 pb_settings='AO DQ CY HU JN LX PR SZ VW FI',
 sorted_rotors='b254',
 ringstellung='A'+ind_ch[19]+ind_ch[9]+ind_ch[24],
 reflector='B Thin'
       )
       
machine_capitan_B.grundstellung('AGCA')

In [ ]:
received_message_capB = machine_capitan_B.encipher_text(capB_message)

In [ ]:
received_message_capB

ICHVERTRAUEALLMEINENMANNERN ->  "Ich vertraue all meinen Männern" (Confío en todos mis hombres)

Comprobamos que es correcto:

In [ ]:
received_message_capB == official_F_message